In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
import numpy as np

from utils import insert_newline_in_middle
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
categories = ['Very important', 'Important', 'Moderately important', 'Slightly important', 'Unimportant']

for c in data.iloc[:,30:60].columns:
    data[c] = pd.Categorical(data[c], categories)

In [ ]:
# Limit to columns of interest
df = data.iloc[:,30:60]

In [ ]:
# Hospital level
a = list()

for c in df.columns:
    b = data.groupby(['Country', 'Hospital Name'])[c].value_counts(normalize=True)
    b.name = c
    a.append(b)

d = pd.concat(a, axis=1)

d = d[d.any(axis=1)]

d.index.names = ['Country', 'Hospital Name', 'Categories']

In [ ]:
# Country level
unique_hospitals = d.reset_index().groupby('Country')['Hospital Name'].nunique()
unique_hospitals.name = 'Country'

e = d.groupby(['Country', 'Categories']).sum()
a = list()

for c in e.columns:
    f = e[c].div(unique_hospitals, level='Country')
    f.name = c
    a.append(f)

g = pd.concat(a, axis=1)

In [ ]:
# EU level
from conversion import population
population = pd.Series(population)
weight = population / population.sum()

m = g.index.get_level_values(0).map(weight)
h = g.multiply(m, axis=0)
i = h.groupby('Categories').sum().T

In [ ]:
result = i
relative = result.div(result.sum(axis=1), axis=0) * 100
relative = relative.round(0)

In [ ]:
relative = relative[['Unimportant', 'Slightly important', 'Moderately important', 'Important', 'Very important']]

In [ ]:
new_index = [insert_newline_in_middle(x) for x in relative.index]
relative.index = new_index

In [ ]:
custom_cmap = LinearSegmentedColormap.from_list('custom_red', ['white', '#2F3088'])

fig, axs = plt.subplots(
    nrows=3,
    figsize=(2,18),
    gridspec_kw={
        'height_ratios':[14, 9, 7]
    },
    sharex=True
)

inputs = relative.iloc[:14]
throughputs = relative.iloc[14:23]
outputs = relative.iloc[23:]

axs[0].imshow(inputs, aspect='auto', cmap=custom_cmap, vmin=0, vmax=100)
axs[0].set_yticks(np.arange(len(inputs.index)))
axs[0].set_yticklabels(inputs.index)
axs[0].set_title('Input')

axs[1].imshow(throughputs, aspect='auto', cmap=custom_cmap, vmin=0, vmax=100)
axs[1].set_yticks(np.arange(len(throughputs.index)))
axs[1].set_yticklabels(throughputs.index)
axs[1].set_title('Throughput')

axs[2].imshow(outputs, aspect='auto', cmap=custom_cmap, vmin=0, vmax=100)
axs[2].set_yticks(np.arange(len(outputs.index)))
axs[2].set_yticklabels(outputs.index);
axs[2].set_title('Output')


# Annotate each cell with percentage
arr = inputs.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        axs[0].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color="black")

# Annotate each cell with percentage
arr = throughputs.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        axs[1].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color="black")

# Annotate each cell with percentage
arr = outputs.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        axs[2].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color="black")

for ax in axs:
    ax.set_xticklabels(relative.columns, rotation=90)
    ax.set_xticks(list(range(0,5)))

plt.savefig(here() / 'output/plots/causes.png', dpi=300, bbox_inches='tight')